### Analyzing Google Trends Data for "random forest" vs. "xgboost"
2021-06-01 to 2022-06-01 <br>
data accessed: 2022-06-01 <br>
Google Trends (https://www.google.com/trends)

In [9]:
import numpy as np
import pandas as pd

skiprows = 2

In [10]:
path = "./data/20220601_random_forest_xgboost.csv"
data = pd.read_csv(path, skiprows=skiprows)
data.columns = ["week", "random forest", "xgboost"]
print(f""""random forest": {np.mean(data["random forest"])}""")
print(f""""xgboost": {np.mean(data["xgboost"])}""")

"random forest": 70.4423076923077
"xgboost": 47.67307692307692
